In [26]:
#import adafruit_dht
import argparse
import numpy as np
import time
import tensorflow as tf
from scipy import signal
#from board import D4

In [15]:
converter = tf.lite.TFLiteConverter.from_saved_model('E:\Github\Machine-learning-for-IOT\Lab3\ex2\MLP')
tflite_model = converter.convert()
tflite_model_dir = 'E:\Github\Machine-learning-for-IOT\Lab3\ex2\MLP\MLP.tflite'
with open(tflite_model_dir, 'wb') as fp:
    fp.write(tflite_model)

In [16]:
interpreter = tf.lite.Interpreter(model_path='E:\Github\Machine-learning-for-IOT\Lab3\ex2\MLP\MLP.tflite')
interpreter.allocate_tensors()

In [17]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [29]:
chunk = 4800
#resolution = pyaudio.paInt16
samp_rate = 48000
record_secs = 1 # seconds to record
dev_index = 0 # device index found by p.get_device_info_by_index(ii)
chunks = int((samp_rate / chunk) * record_secs)
length = int(0.016*16000)
stride = int(0.008*16000)

In [23]:
# buf = BytesIO()

# audio = pyaudio.PyAudio() # create pyaudio instantiation
# stream = audio.open(format=resolution, rate=samp_rate, channels=1,
#                     input_device_index=dev_index, input=True,
#                     frames_per_buffer=chunk)
# stream.stop_stream()

COMMANDS = ['stop', 'up', 'yes', 'right', 'left', 'no', 'silence', 'down', 'go']

In [34]:
while True:



    audio = tf.io.read_file('E:/Github/Machine-learning-for-IOT/Lab3/ex2/data/mini_speech_commands/down/0a9f9af7_nohash_0.wav')
    tf_audio, rate = tf.audio.decode_wav(audio)
    sample = tf.squeeze(tf_audio, 1)
    start = time.time()
    sample = signal.resample_poly(sample, 1, 3)
    sample = tf.convert_to_tensor(sample, dtype=tf.float32)
    stft = tf.signal.stft(sample, length, stride,
            fft_length=length)
    spectrogram = tf.abs(stft)
    spectrogram = tf.reshape(spectrogram, [1, 6, 731, 1])
    spectrogram = tf.image.resize(spectrogram, [32, 32])
    end = time.time()
    preprocessing = (end-start)*1e3
    print('Preprocessing {:.3f}ms'.format(preprocessing))

    start = time.time()
    interpreter.set_tensor(input_details[0]['index'], spectrogram)
    interpreter.invoke()
    predicted = interpreter.get_tensor(output_details[0]['index'])
    end = time.time()
    inference = (end-start)*1e3
    print('Inference {:.3f}ms'.format(inference))
    print('Total {:.3f}ms'.format(preprocessing+inference))
    index = np.argmax(predicted[0])
    print('Command:', COMMANDS[index])
    print()
    time.sleep(0.5)
    break

Preprocessing 8.200ms
Inference 1.000ms
Total 9.200ms
Command: down

